# SELENIUM
## RPA - AUTOMACAO DE CLIQUE EM LINK PÚBLICO
### Fonte de dados: tse.jus.br

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
from selenium.webdriver.chrome.options import Options
import os
import sys

In [3]:
def enable_download_headless(browser, download_dir):
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    params = {'cmd':'Page.setDownloadBehavior', 'params':{'behavior':'allow','downloadPath': download_dir}}
    browser.execute("send_command", params)

In [16]:
def download(url, seletorHtml, tam_arquivoRemoto, endereco_arquivoLocal):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_experimental_option("prefs", {
            "download.default_directory": "c:\TESTE",
            "download.prompt_for_download":  True,
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": True
    })
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-software-rasterizer')

    driver = webdriver.Chrome(executable_path="c:/bigdata/chromedriver.exe", options = chrome_options)
    download_dir = "c:\TESTE"
    enable_download_headless(driver, download_dir)
    
    #driver.implicitly_wait(200)
    driver.get(url)
    time.sleep(5)
    search_input = driver.find_element_by_xpath(seletorHtml)
    tam_ArqLocal = os.stat(endereco_arquivoLocal).st_size
    if search_input:    
        try:
            print('Elemento capturado >>>>> ', search_input.text)
            print('TAMANHO DO ARQUIVO: ',tam_arquivoRemoto)
            search_input.click()
            while tam_arquivoRemoto != tam_ArqLocal:
                print('downloading...')
                print('Remoto ',tam_arquivoRemoto,' / Local', tam_ArqLocal)
            print('Ok, download realizado com sucesso!')
        except:
            print('erro')
    
    return search_input

    

In [17]:
url = 'https://www.tse.jus.br/hotsites/pesquisas-eleitorais/candidatos.html'
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

## so pegar de 2010 em diante

In [18]:
anos = []
for i in soup.findAll('li', class_='span-4'):
    ano = i.find('a').getText()
    if int(ano) >= 2010:
        anos.append(ano)

In [19]:
url_download = 'https://www.tse.jus.br/hotsites/pesquisas-eleitorais/candidatos_anos/'
url_zip = 'http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_'
Xpath = lambda x : '//*[@id="conteudo"]/div[4]/p[1]/a' if x is not None else '//*[@id="conteudo"]/div[4]/p[2]/a'

In [20]:
def varredura(x):
    url = x
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    return Xpath(soup.find('p').find('a'))

def calcTamArquivo(nomeArquivo, ano):
    urlZip = '{}{}.zip'.format(nomeArquivo, ano)
    x = urlopen(urlZip)
    tam = x.info()['Content-length']
    return tam

def nomeArqLocal(nomeArquivo, ano):
    arqLocal = '{}{}.zip'.format(nomeArquivo, ano)
    nomeArqLocal = 'C:/TESTE/' + arqLocal.split('/')[-1]
    return nomeArqLocal
    

In [21]:
for ano in anos:
    url = '{}{}.html'.format(url_download, ano)
    download(url, varredura(url), calcTamArquivo(url_zip, ano), nomeArqLocal(url_zip, ano))    

FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'C:/TESTE/consulta_cand_2010.zip'

In [16]:
print(nomeArqLocal(url_zip, '2012'))

C:/TESTE/consulta_cand_2012.zip
